# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession
se = SparkSession(sc)

import pyspark.sql.functions as F
from pyspark.sql.window import Window

from operator import add

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-19 15:20:00,527 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
sc

<SparkContext master=yarn appName=jupyter>

In [3]:
sc.getConf().get('spark.executor.instances')

'2'

In [4]:
se

In [5]:
! hdfs dfs -ls -h /data

Found 2 items
-rw-r--r--   1 root supergroup     30.7 M 2023-09-24 20:38 /data/clickstream.csv
drwxr-xr-x   - root supergroup          0 2023-09-24 20:38 /data/transactions


In [6]:
! hdfs dfs -head /data/clickstream.csv

user_id	session_id	event_type	event_page	timestamp
562	507	page	main	1695584127
562	507	event	main	1695584134
562	507	event	main	1695584144
562	507	event	main	1695584147
562	507	wNaxLlerrorU	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584160
562	507	page	rabota	1695584166
562	507	event	rabota	1695584174
562	507	event	rabota	1695584181
562	507	event	rabota	1695584189
562	507	page	main	1695584194
562	507	event	main	1695584204
562	507	event	main	1695584211
562	507	event	main	1695584211
562	507	event	main	1695584219
562	507	page	bonus	1695584221
562	507	page	online	1695584222
562	507	event	online	1695584230
3539	849	page	main	1695584238
3539	849	event	main	1695584252
3539	849	page	online	1695584261
3539	849	page	bonus	1695584269
3539	849	event	bonus	1695584278
3539	849	page	news	1695584285
3539	849	page	main	1695584291
3539	849	event	main	1695584301
3539	849	page	news	1695584306
3539	849	event	news	1695584307
3539	849	page	vklad	169558

In [7]:
clicks = (
    se.read
    .csv('hdfs:///data/clickstream.csv', sep='\t', header=True, inferSchema=True)
    #.repartition(2, 'user_id')
    #.sort(['user_id', 'session_id', 'timestamp', 'event_type'])
)
clicks.limit(10).show()

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|   4645|       935|      page|   archive|1696270588|
|   2251|       378|      page|    rabota|1696270591|
|   2222|       704|      page|    rabota|1696270600|
|   2366|       339|      page|     vklad|1696270605|
|   2233|       165|     event|    rabota|1696270605|
|   3253|       528|     event|      news|1696270612|
|    109|       433|     event|     bonus|1696270613|
|   4367|       106|     event|  internet|1696270619|
|   2222|       704|     event|    rabota|1696270628|
|   4304|       640|     event|  internet|1696270636|
+-------+----------+----------+----------+----------+



In [8]:
clicks.dtypes

[('user_id', 'int'),
 ('session_id', 'int'),
 ('event_type', 'string'),
 ('event_page', 'string'),
 ('timestamp', 'int')]

In [9]:
clicks.registerTempTable('clicks')

## SQL

In [10]:
result_sql = se.sql('''
WITH add_error_flg as (
    SELECT
        user_id,
        session_id,
        event_page,
        timestamp,
        MAX(CASE WHEN event_type LIKE "%error%" THEN 1 ELSE 0 END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as error_flg,
        LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as last_page
    FROM clicks
),
collect_route AS (
    SELECT
        user_id,
        session_id,
        ROW_NUMBER() OVER (PARTITION BY user_id, session_id ORDER BY timestamp DESC) as rn,
        COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as route_list
    FROM add_error_flg
    WHERE 1=1
        AND error_flg = 0
        AND (last_page != event_page OR last_page IS NULL)
),
agg_route AS (
    SELECT
        user_id,
        session_id,
        CONCAT_WS("-", route_list) as route
    FROM collect_route
    WHERE rn = 1
)
SELECT
    route,
    COUNT(session_id) as count
FROM agg_route
GROUP BY route
ORDER BY count DESC
;
''')

result_sql.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

In [11]:
with open('result_sql.csv', 'w') as file:
    file.write(result_sql.limit(30).toPandas().to_csv(sep='\t', index=False))

## DataFrame

In [12]:
window = Window.partitionBy('user_id', 'session_id')

add_error_df = (
    clicks
    .withColumn(
        'error_flg',
        F.max(
            F.when(
                F.col('event_type').like('%error%'), 1).otherwise(0)
        ).over(window.orderBy('timestamp'))
    )
    .withColumn(
        'last_page',
        F.lag('event_page').over(window.orderBy('timestamp'))
    )
)

collect_route_df = (
    add_error_df
    .filter('error_flg = 0 and (last_page != event_page or last_page is null)')
    .withColumn(
        'rn',
        F.row_number().over(window.orderBy(F.desc('timestamp')))
    )
    .withColumn(
        'route_list',
        F.collect_list('event_page').over(window.orderBy('timestamp'))
    )
    .filter('rn = 1')
)

agg_route_df = (
    collect_route_df
    .select(
        'user_id',
        'session_id',
        F.concat_ws('-', 'route_list').alias('route')
    )
)

result_df = (
    agg_route_df
    .groupBy('route')
    .agg(F.count('session_id').alias('count'))
    .orderBy(F.desc('count'))
)

result_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

In [13]:
with open('result_df.csv', 'w') as file:
    file.write(result_df.limit(30).toPandas().to_csv(sep='\t', index=False))

## RDD

In [14]:
clicks_rdd = sc.textFile("hdfs:/data/clickstream.csv").map(lambda line: line.split("\t"))
clicks_rdd.take(3)

[['user_id', 'session_id', 'event_type', 'event_page', 'timestamp'],
 ['562', '507', 'page', 'main', '1695584127'],
 ['562', '507', 'event', 'main', '1695584134']]

In [15]:
filtered_rdd = (
    clicks_rdd
    .filter(lambda x: x[0] != 'user_id')
    .map(lambda x: ((int(x[0]), int(x[1])), (int(x[4]), x[2], x[3])))
    .partitionBy(2, partitionFunc=lambda x: x[0] % 2)
    #.sortBy(lambda x: (x[0][0], x[0][1], x[1][0]))
)
filtered_rdd.take(3)

[((1834, 461), (1698218835, 'event', 'vklad')),
 ((1868, 806), (1698218843, 'page', 'main')),
 ((2094, 355), (1698218844, 'event', 'bonus'))]

In [16]:
def sort_filter_pages(values):
    timestamps = []
    pages = []
    error_timestamp = 3e9
    for timestamp, event_type, event_page in values:
        if "error" in event_type:
            error_timestamp = min(error_timestamp, timestamp)
            continue
        timestamps.append(timestamp)
        pages.append(event_page)

    filtered_pages = []
    last_page = None
    for timestamp, page in sorted(zip(timestamps, pages)):
        if timestamp >= error_timestamp:
            break
        if page != last_page:
            filtered_pages.append(page)
            last_page = page
            
    return '-'.join(filtered_pages)

In [17]:
result_rdd = (
    filtered_rdd
    .groupByKey()
    .mapValues(sort_filter_pages)
    .map(lambda x: (x[1], 1))
    .reduceByKey(add)
    .sortBy(lambda x: x[1], ascending=False)
)
result_rdd.take(30)

[('main', 8184),
 ('main-archive', 1112),
 ('main-rabota', 1047),
 ('main-internet', 895),
 ('main-bonus', 869),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 138),
 ('main-bonus-rabota', 135),
 ('main-news-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-tariffs-internet', 114),
 ('main-news-archive', 113),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 104),
 ('main-internet-news', 103),
 ('main-tariffs-archive', 103),
 ('main-rabota-main', 94)]

The order of routes in RDD approach is the same as in SQL/DF approaches. There is some inaccuracy in the count column (about +/- 1 observation for few routes), but in my opinion, this situation doesn't affect the results.

In [18]:
text = ['route\tcount\n']
text.extend(['\t'.join([line[0], str(line[1])]) + '\n' for line in result_rdd.take(30)])
with open('result_rdd.csv', 'w') as file:
    file.writelines(text)

In [19]:
# sc.stop()

In [20]:
# import pandas as pd

# df = pd.read_csv('result_df.csv', sep='\t')
# rdd = pd.read_csv('result_rdd.csv', sep='\t')

# joined = df.merge(rdd, on='route', suffixes=['_df', '_rdd'])
# joined['difference'] = joined.count_df - joined.count_rdd
# round((joined.difference.sum() / joined.count_df.sum()), 4)

In [21]:
! pwd

/home/jovyan/work
